In [79]:
import pandas as pd
import numpy as np

In [80]:
df = pd.read_csv('Final_crop_yield_dataset.csv')
df

,Date,Crop_Type,Soil_Type,Soil_pH,Temperature,Humidity,Wind_Speed,N,P,K,Crop_Yield,Soil_Quality
0,2014-01-01,Wheat,Peaty,5.50,9.440599,80.000000,10.956707,60.5,45.0,31.5,0.000000,22.833333
1,2014-01-01,Corn,Loamy,6.50,20.052576,79.947424,8.591577,84.0,66.0,50.0,104.871310,66.666667
2,2014-01-01,Rice,Peaty,5.50,12.143099,80.000000,7.227751,71.5,54.0,38.5,0.000000,27.333333
3,2014-01-01,Barley,Sandy,6.75,19.751848,80.000000,2.682683,50.0,40.0,30.0,58.939796,35.000000
4,2014-01-01,Soybean,Peaty,5.50,16.110395,80.000000,7.696070,49.5,45.0,38.5,32.970413,22.166667
...,...,...,...,...,...,...,...,...,...,...,...,...
36515,2023-12-31,Cotton,Clay,6.25,19.538555,80.000000,3.666664,66.0,55.0,48.0,73.323885,49.291667
36516,2023-12-31,Sugarcane,Peaty,5.50,21.068336,78.931664,8.795036,71.5,54.0,42.0,39.226521,27.916667
36517,2023-12-31,Tomato,Sandy,6.75,6.030148,80.000000,9.409497,50.0,36.0,30.0,0.000000,33.833333
36518,2023-12-31,Potato,Peaty,5.50,11.079561,80.000000,10.969366,60.5,45.0,31.5,6.067881,22.833333


In [81]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor

In [82]:
df.isnull().mean()*100

Date            0.0
Crop_Type       0.0
Soil_Type       0.0
Soil_pH         0.0
Temperature     0.0
Humidity        0.0
Wind_Speed      0.0
N               0.0
P               0.0
K               0.0
Crop_Yield      0.0
Soil_Quality    0.0
dtype: float64

In [83]:
df.duplicated().sum()

0

In [84]:
df = df.drop(columns=['Date','Soil_Type','Soil_Quality','Wind_Speed','N','P','K'])

In [85]:
df

,Crop_Type,Soil_pH,Temperature,Humidity,Crop_Yield
0,Wheat,5.50,9.440599,80.000000,0.000000
1,Corn,6.50,20.052576,79.947424,104.871310
2,Rice,5.50,12.143099,80.000000,0.000000
3,Barley,6.75,19.751848,80.000000,58.939796
4,Soybean,5.50,16.110395,80.000000,32.970413
...,...,...,...,...,...
36515,Cotton,6.25,19.538555,80.000000,73.323885
36516,Sugarcane,5.50,21.068336,78.931664,39.226521
36517,Tomato,6.75,6.030148,80.000000,0.000000
36518,Potato,5.50,11.079561,80.000000,6.067881


In [86]:
df['Crop_Type'].value_counts()

Crop_Type
Wheat        3652
Corn         3652
Rice         3652
Barley       3652
Soybean      3652
Cotton       3652
Sugarcane    3652
Tomato       3652
Potato       3652
Sunflower    3652
Name: count, dtype: int64

In [87]:
df['Crop_Type'].nunique()

10

In [88]:
df.columns

Index(['Crop_Type', 'Soil_pH', 'Temperature', 'Humidity', 'Crop_Yield'], dtype='object')

In [89]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test ,y_train, y_test = train_test_split(X,y,test_size=0.2)

In [92]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((29216, 4), (7304, 4), (29216,), (7304,))

In [93]:
X_train.columns

Index(['Crop_Type', 'Soil_pH', 'Temperature', 'Humidity'], dtype='object')

In [94]:
transformer = ColumnTransformer(transformers=[
    ('trf1',StandardScaler(),['Soil_pH', 'Temperature', 'Humidity']),
    ('trf2',OneHotEncoder(sparse=False,drop='first'),['Crop_Type'])
],remainder='passthrough')

In [95]:
X_train_trf = transformer.fit_transform(X_train)
X_test_trf = transformer.transform(X_test)

C:\Users\HP TECHNOLOGY\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [96]:
dt = DecisionTreeRegressor()
dt.fit(X_train_trf,y_train)

DecisionTreeRegressor()

In [97]:
y_pred = dt.predict(X_test_trf)

from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.9589648944048388

In [98]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train_trf, y_train)
y_pred2 = rf.predict(X_test_trf)

from sklearn.metrics import r2_score

r2_score(y_test, y_pred2)

0.9718435146867089